# [Trinnect] HFS Room Selection Refresher
##  Install the support package and chrome webdriver to continue.
## Also, fill in the SMTP information for e-mail.
## No warranty, no guide, no anything. Use it in your own risk.

In [9]:
# Import Statements
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from urllib.request import urlopen
from email.header import Header
from email.mime.text import MIMEText
import smtplib
import time
import datetime
import random
import ctypes
import winsound

# Define mailing destinations.
mailing_email = ''  # Example: 'excited@elder.cn'
mailing_password = ''  # Example: 'high_level_19260817'
mailing_list = []  # Leave this blank
smtp_server = 'smtp-mail.outlook.com'  # Example: 'smtp-mail.outlook.com' (For Microsoft)
smtp_port = 587  # Example: 587 (For Microsoft)

# Mailing list
mailing_list.append('')  # Example: 'baohua_zhang@naive.hk'
mailing_list.append('')  # Example: 'jianhua_dong@hongkong.hk'
mailing_list.append('')
mailing_list.append('')


# Define a messageBox to pop up.
def message_box(title, text, style):
    return ctypes.windll.user32.MessageBoxW(0, text, title, style)


# Define a function for beeping
# Beep five times to indicate expire credentials.
def beep(beep_count):
    temp_beep_count = beep_count
    while temp_beep_count is not 0:
        winsound.Beep(600, 1000)
        temp_beep_count -= 1


# Define a function to establish connection with SMTP server.
def connect_smtp():
    server = smtplib.SMTP(smtp_server, smtp_port)
    server.connect(smtp_server, smtp_port)
    server.ehlo()
    server.starttls()
    server.ehlo()
    server.set_debuglevel(1)
    server.login(mailing_email, mailing_password)



In [10]:
# This driver will call HFS. Log into HFS and navigate to search page.
driver = webdriver.Chrome()
hfs_url = 'https://ucharm.hfs.washington.edu/ucharm/SelectRoomResults.asp'
driver.get(hfs_url)

In [11]:
# Initialize variables
hasSpot = False
hasDataTable = False
stopLoop = False
countTime = True
text = ''
stopTime = 0
timeElapsed = 0
timeToStop = 30000


# Main program
while stopLoop is not True:
        
    # Detect error on page
    try:
        # See if the page still pops out error.
        text = driver.find_elements_by_class_name('Error')
        data_table = driver.find_element_by_class_name('DataTable')
        
        # Or see if my login is void
        ifExpired = driver.find_element_by_class_name('login')
        message_box('Request Login', 'You need to login.', 1)
        
        # Stop if credential expired
        if len(ifExpired) != 0:
            # Beep five times to indicate expire credentials.
            beep(5)
                
            # Stop if my login is void.
            stopLoop = True  
            
    # Stop the program from throw exception.
    except NoSuchElementException:
        pass  
    
    # If still does not have space, keep refreshing
    if len(text) != 0:
        # Halt a certain time, so it looks more natural
        time.sleep(stopTime)  
        
        # And refresh the page.
        driver.refresh()  
        
        # Stop if time elapsed equal to time to stop
        if timeElapsed == timeToStop:
            stopLoop = True
            
        # Get current time
        hour = datetime.datetime.now().hour
        # Refreshes slower in midnight and morning
        if 1 < hour < 7:
            stopTime = random.randint(600, 1200)  
        else:
            # Refreshes faster during the day
            stopTime = random.randint(300, 600)
            
        # Update elapsed time so it keeps counting.
        timeElapsed += stopTime 
        
    # Otherwise if there is a space, execute the following responses.
    else:
        # Print a box
        message_box('Program Ended', 'The program timed out or found an apartment', 1) 
        
        # If there exists a data table (There is indeed rooms)
        if len(data_table) != 0:
            # Connect to SMTP
            connect_smtp()
            
            # Initialize amount of mail sent.
            mail_sent = 0
            
            # Send out email to those who needs to be notified.
            while mail_sent < len(mailing_list):
                
                # Information of notify email.
                msg = MIMEText('There is at least one apartment available.', 'plain', 'utf-8')
                msg['From'] = mailing_email
                msg['To'] = mailing_list[mail_sent]
                msg['Subject'] = Header(u'[Notify.Trinnect] There is an apartment available.', 'utf-8').encode()
                server.sendmail(mailing_email, [mailing_list[mail_sent]], msg.as_string())
                mail_sent += 1
            
            # Close server connection
            server.quit()
            
            # Beep three times
            beep(3)
            
            # Stop refreshing
            countTime = False   
            stopLoop = True
